In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from time import time
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.ensemble import BaggingRegressor

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  

# 一、读取训练集和测试集，确定训练（预测）目标

In [2]:
train = pd.read_csv('train_modified_Gaoshou.csv')
#train = pd.read_csv('train_modified.csv')
#test = pd.read_csv('test_modified.csv')

from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

IOError: File train_modified_Gaoshou.csv does not exist

target='SalePrice'
IDcol = 'ID'

In [ ]:
#df_train_target = train['SalePrice']
#df_train_data = train.drop(['SalePrice'],axis = 1)
#print 'df_train_data shape is ', df_train_data.shape
#print 'df_train_target shape is ', df_train_target.shape
#print 'df_train_target shape is ', test.shape


#df_train_target = train['SalePrice'].values
df_train_target = train['SalePrice']
df_train_data = train.drop(['SalePrice'],axis = 1).values
#df_train_target["SalePrice"] = np.log(df_train_target["SalePrice"])



print 'df_train_data shape is ', df_train_data.shape
print 'df_train_target shape is ', df_train_target.shape

# 二、切分训练集，初步尝试各种回归模型

In [ ]:
# 总得切分一下数据咯（训练集和测试集）
cv = cross_validation.ShuffleSplit(len(df_train_data), n_iter=3, test_size=0.2,
    random_state=0)



# 各种模型来一圈

print "Ridge"    
z0=list()
for train, test in cv:    
    svc0 = linear_model.Ridge().fit(df_train_data[train], df_train_target[train])
    
    print("Ridge score on training set: ", rmse(svc0.predict(df_train_data[test]), df_train_target[test]))
    z0.append(rmse(svc0.predict(df_train_data[test]), df_train_target[test]))
    
print "Lasso"  
z1=list()
for train, test in cv:    
    svc1 = Lasso(alpha=0.00099, max_iter=50000).fit(df_train_data[train], df_train_target[train])
    
    print("Lasso score on training set: ", rmse(svc1.predict(df_train_data[test]), df_train_target[test]))
    z1.append(rmse(svc1.predict(df_train_data[test]), df_train_target[test]))

    
    
print "GradientBoostingRegressor"
for train, test in cv:
    
    svc2 = GradientBoostingRegressor().fit(df_train_data[train], df_train_target[train])
    print("GradientBoostingRegressor score on training set: ", rmse(svc2.predict(df_train_data[test]), df_train_target[test]))

print "ExtraTreesRegressor"
for train, test in cv:
    
    svc3 = ExtraTreesRegressor().fit(df_train_data[train], df_train_target[train])
    print("ExtraTreesRegressor score on training set: ", rmse(svc3.predict(df_train_data[test]), df_train_target[test]))
        
    
print "随机森林回归/Random Forest(n_estimators = 100)"    
for train, test in cv:    
    svc4 = RandomForestRegressor(n_estimators = 100).fit(df_train_data[train], df_train_target[train])
    print("RF score on training set: ", rmse(svc4.predict(df_train_data[test]), df_train_target[test]))

print "XGBRegressor"    
for train, test in cv:  
    z5=list()
    svc5 =  XGBRegressor(colsample_bytree=0.2,
                 gamma=0.0,
                 learning_rate=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                 
#                 reg_alpha=0.9,
#                 reg_lambda=0.6,
                 subsample=0.2,
                 seed=42,
                 silent=1).fit(df_train_data[train], df_train_target[train])
    print("XGBoost score on training set: ", rmse(svc5.predict(df_train_data[test]), df_train_target[test]))
    z5.append(rmse(svc5.predict(df_train_data[test]), df_train_target[test]))
    
print "随机森林回归/Random Forest(n_estimators = 100)"    
for train, test in cv:    
    svc6 = BaggingRegressor().fit(df_train_data[train], df_train_target[train])
    print("RF score on training set: ", rmse(svc6.predict(df_train_data[test]), df_train_target[test]))




In [ ]:
s015 = rmse((svc0.predict(df_train_data)+svc1.predict(df_train_data)+svc5.predict(df_train_data))/3, df_train_target)
s01 = rmse((svc0.predict(df_train_data)+svc1.predict(df_train_data))/2, df_train_target)
s05 = rmse((svc0.predict(df_train_data)+svc5.predict(df_train_data))/2, df_train_target)
s15 = rmse((svc1.predict(df_train_data)+svc5.predict(df_train_data))/2, df_train_target)






print "s015:%s,s01:%s,s15:%s,s05:%s, " %(s015,s01,s15,s05)                                                      

In [ ]:
#将多个模型的结果融合，再跑一边xgboost
svc0 = linear_model.Ridge().fit(df_train_data, df_train_target)

svc1 = Lasso(alpha=0.00099, max_iter=50000).fit(df_train_data, df_train_target)

svc2 = GradientBoostingRegressor().fit(df_train_data, df_train_target)

svc3 = ExtraTreesRegressor().fit(df_train_data, df_train_target)

svc4 = RandomForestRegressor(n_estimators = 100).fit(df_train_data, df_train_target)

svc5 =  XGBRegressor(max_depth=3, 
                    learning_rate=0.1, 
                    n_estimators=100, 
                    silent=True, 
                    objective='reg:linear', 
                    nthread=-1, gamma=0, 
                    min_child_weight=1, 
                    max_delta_step=0, 
                    subsample=1, 
                    colsample_bytree=1, 
#                    colsample_bylevel=1, 
#                    reg_alpha=0, 
#                    reg_lambda=1, 
#                    scale_pos_weight=1, 
                    base_score=0.5, 
                    seed=0, 
                    missing=None).fit(df_train_data, df_train_target)






In [ ]:
trai = pd.read_csv('train_modified_Gaoshou.csv')
df1 = trai.drop(['SalePrice'],axis = 1)
#df1 = pd.read_csv('test_modified.csv')

df1_train = pd.DataFrame({
                    'svc0':svc0.predict(df1),
#                    'svc1':svc1.predict(df1),
#                    'svc2':svc2.predict(df1),
#                    'svc3':svc3.predict(df1),
#                    'svc4':svc4.predict(df1),
                    'svc5':svc5.predict(df1),
                   })
df1_train = df1_train.values

svc_last =  BaggingRegressor().fit(df1_train[train], df_train_target[train])

cv1 = cross_validation.ShuffleSplit(len(df_train_target), n_iter=10, test_size=0.2,
    random_state=0)

for train, test in cv1:    
    svc_last1 =  svc_last
    print rmse(svc_last1.predict(df1_train[test]), df_train_target[test])




# 三、将各个具有较好默认表现的回归模型，进行更为细致的grid调参和CV

In [ ]:
X = df_train_data
y = df_train_target

X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.2, random_state=0)

tuned_parameters = [{
        'alpha':[1.0,2.0,3.0,4.0,5.0,6.0], 
       'tol':[1.0,2.0,3.0,4.0,5.0], 
#        'max_iter':[3,5,7],

#        'n_estimators':[100,200,500,1000,2000], 
#        'subsample':[1,2,5], 
#        'colsample_bytree':[1,2,5], 
                    }]   
    
scores = ['r2']

In [ ]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_train_data, df_train_target, test_size=0.3, random_state=0)

# TODO：创建你希望调整的参数列表
parameters = {'alpha':[1.0,2.0,3.0,4.0,5.0,6.0], 
       'tol':[1.0,2.0,3.0,4.0,5.0], }

# TODO：初始化分类器
clf = linear_model.Ridge()

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(mean_squared_error)

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf,param_grid=parameters,cv=None,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print clf

# 四、通过学习曲线，验证模型是否过拟合或者欠拟合

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):
    
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt



cv = cross_validation.ShuffleSplit(df_train_data.shape[0], n_iter=10,test_size=0.2, random_state=0)
#estimator = linear_model.Ridge( copy_X=True, fit_intercept=False, max_iter=None,
#      normalize=False, random_state=0, solver='auto',alpha=2.0,tol=1.0)
                                  
#plot_learning_curve(estimator, title, X, y, (0.0, 1.01), cv=cv, n_jobs=4)


estimator0 = linear_model.Ridge()
estimator1 = Lasso(alpha=0.00099, max_iter=50000)
estimator2 = GradientBoostingRegressor()
estimator3 = ExtraTreesRegressor()
estimator4 = RandomForestRegressor()
estimator5 = XGBRegressor()


title = "Learning Curves (Ridge)"
plot_learning_curve(estimator0, title, X, y, (0.0, 1.01), cv=cv, n_jobs=4)
title = "Learning Curves (Lasso)"
plot_learning_curve(estimator1, title, X, y, (0.0, 1.01), cv=cv, n_jobs=4)


title = "Learning Curves (XGBRegressor)"
plot_learning_curve(estimator5, title, X, y, (0.0, 1.01), cv=cv, n_jobs=4)

plt.show()  


plt.show()

# 五、用得到的参数，训练测试集，得到结果并保存

df2 = pd.read_csv('test_modified.csv')


df2_train = pd.DataFrame({
                    'svc0':svc0.predict(df2),
                    'svc1':svc1.predict(df2),
                    'svc2':svc2.predict(df2),
                    'svc3':svc3.predict(df2),
                    'svc4':svc4.predict(df2),
#                    'SalePrice':df_train_target,
                   })


df2_train = df2_train.values

result_last_var = svc_last.predict(df2_train)


result_last = pd.DataFrame({'Id':df2['Id'],'SalePrice':result_last_var})

In [ ]:
df2 = pd.read_csv('test_modified_Gaoshou.csv')

pre0 = svc0.predict(df2)
pre1 = svc1.predict(df2)
pre5 = svc5.predict(df2)

predicts = (pre1+pre5)/2

In [ ]:
#加第二层
df1_train = pd.DataFrame({
                    'svc0':svc0.predict(df2),
#                    'svc1':svc1.predict(df2),
#                    'svc2':svc2.predict(df1),
#                    'svc3':svc3.predict(df1),
#                    'svc4':svc4.predict(df1),
                    'svc5':svc5.predict(df2),
                   })
#df1_train = df1_train.values

predicts = svc_last.predict(df1_train)
#predicts = np.exp(predict1)

In [ ]:
predicts

In [ ]:

predicts_end = np.exp(predicts)

In [ ]:
df3 = pd.read_csv('test_modified.csv')
result_last = pd.DataFrame({'Id':df3['Id'],'SalePrice':predicts_end})

In [ ]:
result_last.to_csv('12-11.csv',index=False)

In [ ]:
result_last.describe()

In [ ]:
result_last.info()

In [ ]:
aaaaaaa= pd.read_csv('output.csv')
aaaaaaa.describe()

In [ ]:


plt.hist(result_last['SalePrice'], bins=12, normed=True, color="#FF0000", alpha=.9)
plt.hist(aaaaaaa['SalePrice'], bins=12, normed=True, color="#C1F320", alpha=.9)
plt.hist(np.exp(df_train_target), bins=12, normed=True,  alpha=.9)



In [ ]:
#ensemble函数
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]

            S_test[:, i] = S_test_i.mean(1)

        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred